In [ ]:
# App https://play.google.com/store/apps/details?id=droidhang.twgame.restaurant&hl=en_US&gl=US

# General Includes
import subprocess, json,os
from tqdm import tqdm
from IPython.display import clear_output,display
import time
from pathlib import Path
import random

In [ ]:
class StateMachine():
    def __init__(self,name):
        self.file = Path(f'{name}.StateMachine.json')
        if not self.file.exists():
            self.file.write_text(json.dumps({
                'name': name,
                'initialState':None,
                'currentState':None,
                'rules':{}
            }))
        self.state = json.loads(self.file.read_text())
        if self.state['initialState'] is not None:
            self.currentState = self.state['rules'][self.state['initialState']]
        
        
    def setupInitalTo(self,name):
        self.state['initialState'] = name
        self.file.write_text(json.dumps(self.state))
        self.currentState = self.state['rules'][self.state['initialState']]
        
    def addState(self, state_name,to):
        self.state['rules'][state_name] = {
            'name':state_name,
            'to':to
        }
        self.file.write_text(json.dumps(self.state))
    def resetState(self):
        self.currentState = self.state['rules'][self.state['initialState']]
        self.state['currentState'] = self.state['initialState']
        self.file.write_text(json.dumps(self.state))
        
    def moveState(self, new_state):
        if new_state in self.currentState['to']:
            self.currentState = self.state['rules'][new_state]
            self.state['currentState'] = new_state
            self.file.write_text(json.dumps(self.state))
        else:
            raise Exception("Bad State Move")
    
    

In [ ]:
test = StateMachine('Mining and Smelting')
test.addState("Mine",['DoMining','checkInventory','teleportToSmelting'])
test.addState("teleportToMining",['Mine'])
test.addState("teleportToSmelting",['Smelt'])
test.addState("checkInventory",['Mine','teleportToSmelting','Bank'])
test.addState("teleportToSmelting",['smelt',"checkInventory"])
test.addState("Bank",['DoBanking','teleportToMining'])
test.addState("Smelt",['DoSmelting','teleportToMining'])
test.setupInitalTo('Mine')

In [ ]:
test.moveState('checkInventory')
test.currentState
test.resetState()

In [ ]:
def shell(cmd,debug=False,returns_POpen = False,close_fds=False):
    if debug: print(cmd)
    
    if returns_POpen:
        process = subprocess.Popen(cmd, close_fds=close_fds)
        return process
    
    process = subprocess.run(cmd, stdout=subprocess.PIPE)
    return process.stdout.decode('utf-8').strip().splitlines()


def adb(cmd,debug = False,returns_POpen = False,close_fds=False):
    # This function runs adb commands on your connected device or emulator.
    if type(cmd) == str:
        cmd = cmd.split(' ')
    cmd = ['adb'] + cmd
    return shell(
        cmd,
        debug=debug,
        returns_POpen=returns_POpen,
        close_fds=close_fds
    )
adb('wait-for-device')

In [ ]:
def FindColor(im,color,tolerence):
    min_r = color[0]-tolerence
    min_g = color[1]-tolerence
    min_b = color[2]-tolerence

    max_r = color[0]+tolerence
    max_g = color[1]+tolerence
    max_b = color[2]+tolerence

    for x in range(1,im.width-1,1):
        for y in range(1,im.height-1,1):
            rgb = im.getpixel((x,y))
            if (rgb[0] > min_r and rgb[0] < max_r and
                rgb[1] > min_g and rgb[1] < max_g and
                rgb[2] > min_b and rgb[2] < max_b
               ):
                yield(x,y)

In [ ]:
from PIL import Image, ImageDraw
def pullImage():
    adb('shell screencap -p /sdcard/screen.png')
    adb('pull /sdcard/screen.png ./game.png')
    adb("shell rm /sdcard/screen.png")
    im = Image.open("game.png")
    rgb_im = im.convert('RGB')
    return rgb_im

In [ ]:
def getCenter(points,area_range=5):
    x = [p[0] for p in points]
    y = [p[1] for p in points]
    return (int(sum(x) / len(points))+random.randint(0-area_range,area_range), int(sum(y) / len(points))+random.randint(0-area_range,area_range))

In [ ]:
last_click = time.time()
click_throttle = 10
resize_ratio=0.25
can_click = True
color = (173,133,153)

In [ ]:
while True:
    a = pullImage()
    #a = a.resize((int(a.width*resize_ratio),int(a.height*resize_ratio)), Image.Resampling.LANCZOS)
    #clear_output()
    #display(a)
    c = list(FindColor(a,color,10))
    if len(c)>2:
        center = getCenter(c)
        if time.time()-last_click > click_throttle:
            last_click = time.time()
            print(center)
            if can_click:
                adb(f"shell input tap {center[0]} {center[1]}")
                time.sleep(random.choice([2.5,7,14,2,1,0]))
    time.sleep(5)

In [ ]:
display(a)

In [ ]:
a.width

In [ ]:
a.height